# **UB** : Save and Restore Crystal Orientation

see: https://github.com/bluesky/hklpy/issues/50

**Objectives**

1. Save the information defining the crystal orientation into the descriptor
1. Restore crystal orientation from a given Bluesky run
1. List runs that have orientation that can be restored

In [1]:
import gi
gi.require_version('Hkl', '5.0')

from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
import bluesky.plans as bp
import bluesky.plan_stubs as bps
import bluesky.preprocessors as bpp
import databroker
import hkl
from hkl.calc import A_KEV
from hkl.util import Lattice
from hkl.geometries import *
import numpy as np
import pyRestTable
from ophyd import Component, Device, EpicsSignal, Signal
from ophyd.signal import AttributeSignal, ArrayAttributeSignal
from ophyd.sim import *

bec = BestEffortCallback()
bec.disable_plots()
cat = databroker.temp().v2

RE = RunEngine({})
RE.subscribe(bec)
RE.subscribe(cat.v1.insert)
RE.md["notebook"] = "UB_save_restore"
RE.md["objective"] = "Demonstrate UB matrix save & restore"

/home/mintadmin/Apps/anaconda/envs/bluesky_2021_1/lib/python3.8/site-packages/databroker/core.py:1856: UserWarning: There are 2 entrypoints for the databroker handler spec 'NPY_SEQ'. They are [EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', Distribution('ophyd', '1.6.0')), EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', None)]. The match EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', Distribution('ophyd', '1.6.0')) has won the race.
  warnings.warn(


-------------

In [2]:
class OrientationMixin(Device):
#     _constraints = Cpt(
#         ArrayAttributeSignal,
#         attr="_constraints_for_databroker",
#         doc="Constraints",
#         write_access=False,
#     )

#     @property
#     def _constraints_for_databroker(self):
#         """
#         Return the constraints for databroker.
        
#         Cannot write a dictionary from bluesky, so make the
#         dictionary into [[k, *v] for k, v in dict.items()].
#         """
#         return [[k, *v] for k, v in self._constraints_dict.items()]

    # TODO: additions for hkl.diffract.Diffractometer
    
    # def __init__(self, *args, **kwargs):
    #     super().__init__(*args, **kwargs)
    pass

In [3]:
def show_component_kinds(device):
    tbl = pyRestTable.Table()
    tbl.labels = "component kind value".split()
    for attr in device.component_names:
        obj = getattr(device, attr)
        tbl.addRow((attr, obj.kind, obj.get()))
    print(tbl)

In [4]:
def get_orientation_diffractometer(base_class, run):
    run_conf = run.primary.config
    findings = []
    # db.v2[-1].primary.config[purple.name].read().to_dict()
    for det_name in run_conf:
        conf = run_conf[det_name].read()
        if f"{det_name}_orientation_attrs" in conf:
            dd = {
                item[len(det_name)+1:]: conf[item].to_dict()["data"][0]
                for item in conf
            }
            if dd["geometry_name"] == base_class:
                # TODO: compare reals and pseudos?  This is a user option.
                findings.append(det_name)
    return findings

In [5]:
def find_orientations(dfrct):
    tbl = pyRestTable.Table()
    tbl.labels = "scan_id command name_match? orientation_det".split()
    for run in cat.values():
        diffractometer_name = dfrct.name in run.primary.config
        scan_id = run.metadata["start"]["scan_id"]
        cmd = run.metadata["start"]["plan_name"]
        dname = get_orientation_diffractometer(dfrct.geometry_name.get(), run)
        
        tbl.addRow((scan_id, cmd, diffractometer_name, dname))
    print(tbl)

In [6]:
def read_orientation(run, det_name):
    run_conf = run.primary.config
    conf = run_conf[det_name].read()
    if f"{det_name}_orientation_attrs" in conf:
        dd = {
            item[len(det_name)+1:]: conf[item].to_dict()["data"][0]
            for item in conf
        }
    else:
        dd = {}
    return dd

-------------

## Build a simulated 4-circle diffractometer

In [7]:
class Fourc(OrientationMixin, SimulatedE4CV):
    pass

fourc = Fourc("", name="fourc")
fourc.energy.put(A_KEV / 1.54)
a0 = 5.4310196
fourc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
fourc.calc.sample.compute_UB(
    fourc.calc.sample.add_reflection(4, 0, 0, (-145.451, 0, 0, 69.0966)),
    fourc.calc.sample.add_reflection(0, 4, 0, (-145.451, 0, 90, 69.0966))
)
fourc.pa()

orange = Fourc("", name="orange")
orange.pa()

===================== ===========================================================================
term                  value                                                                      
===================== ===========================================================================
diffractometer        fourc                                                                      
geometry              E4CV                                                                       
class                 Fourc                                                                      
energy (keV)          8.05092                                                                    
wavelength (angstrom) 1.54000                                                                    
calc engine           hkl                                                                        
mode                  bissector                                                                  
positions           

In [8]:
fourc._constraints.get()

array([['omega', '-180.0', '180.0', '0.0', 'True', 'False'],
       ['chi', '-180.0', '180.0', '0.0', 'True', 'False'],
       ['phi', '-180.0', '180.0', '0.0', 'True', 'False'],
       ['tth', '-180.0', '180.0', '0.0', 'True', 'False']], dtype='<U6')

In [9]:
class Kappa(OrientationMixin, SimulatedK4CV):
    pass

kappa = Kappa("", name="kappa")
kappa.energy.put(A_KEV / 1.54)
a0 = 5.4310196
kappa.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
kappa.calc.sample.compute_UB(
    kappa.calc.sample.add_reflection(4, 0, 0, (55.4507, 0, 90, -69.0966)), 
    kappa.calc.sample.add_reflection(0, 4, 0, (-1.5950, 134.7568, 123.3554, -69.0966))
)
kappa.pa()

===================== =================================================================================
term                  value                                                                            
===================== =================================================================================
diffractometer        kappa                                                                            
geometry              K4CV                                                                             
class                 Kappa                                                                            
energy (keV)          8.05092                                                                          
wavelength (angstrom) 1.54000                                                                          
calc engine           hkl                                                                              
mode                  bissector                                 

In [10]:
class Sixc(OrientationMixin, SimulatedE6C):
    pass

sixc = Sixc("", name="sixc")
sixc.energy.put(A_KEV / 1.54)
a0 = 5.4310196
sixc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
sixc.calc.sample.compute_UB(
    sixc.calc.sample.add_reflection(4, 0, 0, (0, -145.451, 0, 0, 0, 69.0966)),
    sixc.calc.sample.add_reflection(0, 4, 0, (0, -145.451, 90, 0, 0, 69.0966))
)
sixc.pa()

===================== ========================================================================================================
term                  value                                                                                                   
===================== ========================================================================================================
diffractometer        sixc                                                                                                    
geometry              E6C                                                                                                     
class                 Sixc                                                                                                    
energy (keV)          8.05092                                                                                                 
wavelength (angstrom) 1.54000                                                                                  

In [11]:
def scan_all():
    yield from bp.count([noisy_det])
    yield from bp.count([noisy_det, fourc])
    yield from bp.count([noisy_det, fourc, orange, kappa, sixc])
    yield from bp.scan([noisy_det], fourc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, fourc], fourc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det], kappa.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, kappa], kappa.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det], sixc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, sixc], sixc.h, 0.9, 1.1, 2)

In [12]:
_uids = RE(scan_all())



Transient Scan ID: 1     Time: 2021-04-25 11:04:45
Persistent Unique Scan ID: '154b4198-21ac-4826-bea5-66983ec853a4'
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time |  noisy_det |
+-----------+------------+------------+
|         1 | 11:04:45.5 |      0.900 |
+-----------+------------+------------+
generator count ['154b4198'] (scan num: 1)





Transient Scan ID: 2     Time: 2021-04-25 11:04:45
Persistent Unique Scan ID: 'fc59e462-9282-48cf-8698-53fa3adf84e1'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k |    fourc_l |  noisy_det |
+-----------+------------+------------+------------+------------+------------+
|         1 | 11:04:45.7 |      0.000 |      0.000 |      0.000 |      1.097 |
+-----------+------------+------------+------------+------------+------------+
generator count ['fc59e462'] (scan num: 2)





Transient Scan ID: 3     Time

In [13]:
find_orientations(fourc)

======= ======= =========== ===================
scan_id command name_match? orientation_det    
======= ======= =========== ===================
1       count   False       []                 
2       count   True        ['fourc']          
3       count   True        ['orange', 'fourc']
4       scan    False       []                 
5       scan    True        ['fourc']          
6       scan    False       []                 
7       scan    False       []                 
8       scan    False       []                 
9       scan    False       []                 
======= ======= =========== ===================



In [14]:
run = cat[5]
run.primary.config["fourc"].read()

<xarray.Dataset>
Dimensions:                    (dim_0: 6, dim_1: 6, dim_10: 6, dim_11: 21, dim_2: 3, dim_3: 3, dim_4: 3, dim_5: 3, dim_6: 2, dim_7: 3, dim_8: 4, dim_9: 4, time: 2)
Coordinates:
  * time                       (time) float64 1.619e+09 1.619e+09
Dimensions without coordinates: dim_0, dim_1, dim_10, dim_11, dim_2, dim_3, dim_4, dim_5, dim_6, dim_7, dim_8, dim_9
Data variables: (12/21)
    fourc_energy               (time) float64 8.051 8.051
    fourc_energy_units         (time) <U3 'keV' 'keV'
    fourc_energy_offset        (time) int64 0 0
    fourc_geometry_name        (time) <U4 'E4CV' 'E4CV'
    fourc_class_name           (time) <U5 'Fourc' 'Fourc'
    fourc_sample_name          (time) <U7 'silicon' 'silicon'
    ...                         ...
    fourc__hklpy_version       (time) <U25 '0.3.15+163.g1c0c9fc.dirty' '0.3.1...
    fourc__pseudos             (time, dim_7) <U1 'h' 'k' 'l' 'h' 'k' 'l'
    fourc__reals               (time, dim_8) <U5 'omega' 'chi' ... 'phi' 'tth'
    fourc__constraints         (time, dim_9, dim_10) <U23 'omega' ... 'False'
    fourc__mode                (time) <U9 'bissector' 'bissector'
    fourc_orientation_attrs    (time, dim_11) <U19 'orientation_attrs' ... '_...

In [15]:
read_orientation(cat[5], fourc.name)

{'energy': 8.050922077922078,
 'energy_units': 'keV',
 'energy_offset': 0,
 'geometry_name': 'E4CV',
 'class_name': 'Fourc',
 'sample_name': 'silicon',
 'lattice': [5.4310196, 5.4310196, 5.4310196, 90.0, 90.0, 90.0],
 'lattice_reciprocal': [1.1569071316147683,
  1.1569071316147683,
  1.1569071316147683,
  90.00000000000001,
  90.00000000000001,
  90.00000000000001],
 'U': [[-1.2217304763832569e-05, -0.9999999999253688, 0.0],
  [0.0, 0.0, 1.0],
  [-0.9999999999253688, 1.2217304763832569e-05, 0.0]],
 'UB': [[-1.4134287010388982e-05, -1.156907131528427, 7.084099625231898e-17],
  [0.0, 0.0, 1.1569071316147683],
  [-1.156907131528427, 1.4134287010459822e-05, 7.083926530138442e-17]],
 'reflections_details': [{'reflection': {'h': 4.0, 'k': 0.0, 'l': 0.0},
   'flag': 1,
   'wavelength': 1.54,
   'position': {'omega': -145.451, 'chi': 0.0, 'phi': 0.0, 'tth': 69.0966},
   'orientation_reflection': True},
  {'reflection': {'h': 0.0, 'k': 4.0, 'l': 0.0},
   'flag': 1,
   'wavelength': 1.54,
   'po

------------

TODO: show series of scans with different orientation reflections and different **UB**

In [16]:
fourc.show_constraints()

===== ========= ========== ======================= ==== ========
axis  low_limit high_limit value                   fit  inverted
===== ========= ========== ======================= ==== ========
omega -180.0    180.0      -8.97224432640064       True False   
chi   -180.0    180.0      1.3179200548037037e-109 True False   
phi   -180.0    180.0      0.0007000000002791517   True False   
tth   -180.0    180.0      -17.94448865280128      True False   
===== ========= ========== ======================= ==== ========

